<a href="https://colab.research.google.com/github/yttinalee/data-course-sample/blob/main/Week2_A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [ ]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC
'wget' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [ ]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
metadata.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [ ]:
metadata['title'].unique()

array(["Loud 'N Clear&trade; Personal Sound Amplifier",
       'No7 Lift &amp; Luminate Triple Action Serum 50ml by Boots',
       'No7 Stay Perfect Foundation Cool Vanilla by No7', ...,
       'NOW D-Mannose 500 mg - 120 Veg Capsules (Pack of 3)',
       '12 White Feather Shuttlecocks Birdies Badminton Training Match',
       'FESHFEN Scrunchy Scrunchies Synthetic Hair Bun Updo Hair Pieces Ponytail Hair Extensions Wavy Curly Messy Hair Bun Extensions Donut Hair Chignons Piece Wig'],
      dtype=object)

In [ ]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

In [ ]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_trainings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_trainings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_trainings_by_user }
training_users = list(ratings_trainings_by_user.keys())
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

In [ ]:
# # training data中的user有多少也出現在testint data
dic_user = []
for ii in training_users:
  if ii in users:
    dic_user.append(ii)
len(dic_user)

584

In [ ]:
ratings_trainings_by_asin = ratings_testings.groupby('asin').agg(list).reset_index()['asin']
ratings_testings_by_asin = ratings_testings.groupby('asin').agg(list).reset_index()['asin']

In [ ]:
# # training data中的asin有多少也出現在testint data
dic = []
for ii in ratings_trainings_by_asin:
  if ii in ratings_testings_by_asin:
    dic.append(ii)
len(dic)

0

In [ ]:
category = metadata['rank'].str.split(' in ', expand=True)
category = category.rename(columns={category.columns[0]:'ranking', category.columns[1]:'categories'})
# category.head()

In [ ]:
category['ranking'] = category['ranking'].str.replace(',','')
category['categories'] = category['categories'].str.replace('amp;','')
category['categories'] = category['categories'].str.replace('\(', '')
category['ranking'] = pd.to_numeric(category['ranking'])

In [ ]:
category.groupby('categories').count()

,ranking
categories,
Automotive,1
Baby,2
Beauty & Personal Care,32380
"Clothing, Shoes & Jewelry",2
Grocery & Gourmet Food,38
Health & Household,21
Home & Kitchen,1
Sports & Outdoors,3
Tools & Home Improvement,1


In [ ]:
meta_use = metadata[['asin', 'title', 'also_buy', 'brand', 'rank', 'also_view', 'details', 'price', 'description']]

In [ ]:
meta_use.groupby('asin').count()['title'].sort_values(ascending=False)

asin
B00026KYLM    2
B00021DI10    2
B00021CWYO    2
B00021CQTA    2
B00021CPNC    2
             ..
B01172CGQK    1
B01172N21I    1
B01172N2HM    1
B01174YHM4    1
6546546450    1
Name: title, Length: 32488, dtype: int64

In [ ]:
meta_use.groupby('title').count()['asin'].sort_values(ascending=False).head(10)

title
Baubax Women's Bomber Travel Jacket                                                                                                                                                          9
Bourjois Healthy Mix Foundation                                                                                                                                                              7
Derma Roller 0.25mm - Facial Skin Care Tool for Face - Micro Needle Roller - 540 Deluxe Titanium Micro Needles - Titanium Microneedle Roller - Perfect Cosmetic Microneedling Skin Roller    7
Sankuwen 15PCs Wool Makeup Brush Set Tools Toiletry Kit                                                                                                                                      7
2                                                                                                                                                                                            5
BMC Nail Art Water Transfer Tattoo Effe

In [ ]:
meta_use['ranking'] = category['ranking']
meta_use['categories'] = category['categories']

<ipython-input-205-110db4146380>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['ranking'] = category['ranking']
<ipython-input-205-110db4146380>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['categories'] = category['categories']


In [ ]:
meta_use['title'] = meta_use['title'].str.replace('&amp;','')

<ipython-input-206-212f390ff629>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  meta_use['title'] = meta_use['title'].str.replace('&amp;','')


1. Item representation

In [ ]:
len(meta_use['title'].unique()), len(meta_use['title']), meta_use['title'].head(7)

(32300,
 32892,
 0        Loud 'N Clear&trade; Personal Sound Amplifier
 1    No7 Lift  Luminate Triple Action Serum 50ml by...
 2      No7 Stay Perfect Foundation Cool Vanilla by No7
 3    Wella Koleston Perfect Hair Colour 44/44 Mediu...
 4    Lacto Calamine Skin Balance Oil control 120 ml...
 5    Mary Kay Satin Hands Hand Cream Travel MINI Si...
 6    Unique Custom Cast Iron Liner Shader Tattoo Ma...
 Name: title, dtype: object)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import string 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics.pairwise import cosine_similarity

lemmatizer = WordNetLemmatizer()

df = meta_use.drop_duplicates(['asin', 'title', 'brand'])

def preproc(sentence):
# for sentence in range(0, len(meta_use['title'])):
  remove_punctuation = ''.join([i for i in sentence if i not in string.punctuation]).lower()
  tokens = word_tokenize(remove_punctuation)
  result = [lemmatizer.lemmatize(voca) for voca in tokens if not voca in stopwords.words('english')]
  return ' '.join(result)

df['title'].apply(lambda x: preproc(x))
# 計算商品用標題所表示的 tfidf 矩陣

tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['title'])

# 計算商品間的相似程度
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# # 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=2):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# # 利用使用者購買過的商品產生推薦
def recommend_items(items, k):
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res

In [ ]:
df['description']

0        [Loud 'N Clear Personal Sound Amplifier allows...
1        [No7 Lift & Luminate Triple Action Serum 50ml ...
2        [No7 Stay Perfect Foundation now stays perfect...
3                                                       []
4        [Lacto Calamine Skin Balance Daily Nourishing ...
                               ...                        
32887                                                   []
32888                                                   []
32889                                                   []
32890    [Brand new and high quality<br> Enables fast v...
32891                                                   []
Name: description, Length: 32488, dtype: object

In [ ]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    for ii in users:
      recommend_cates = []
      recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == ii]['asin'].tolist())]['title'].tolist(), k)
      
      recommendations[ii] = recommend_cates
    return recommendations
    

In [ ]:
def recommender1(training_data, users=[], k=10):
  '''
  * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
  * users: [] 需要被推薦的使用者
  * k: int 每個使用者需要推薦的商品數
  * recommendations: dict
    {
        使用者一： [推薦商品一, 推薦商品二, ...],
        使用者二： [...], ...
    }
  '''
  # training_data = training_data[(training_data['DATE'] >= '2018-06-30') &  (training_data['DATE'] <= '2018-09-01')]
  recommendations = {}
  # n=0
  for ii in users:
    user = training_data[training_data['reviewerID'] == str(ii)]
    # print(len(user))
    recommend_cates = []
    if len(user) > 0:
      prod = user.groupby('asin').count()['overall'].sort_values(ascending=False).reset_index()
      brand = meta_use[meta_use['asin'] == prod['asin'].unique()[0]].sort_values('ranking', ascending=False)
      if len(brand) > 0 and brand['brand'].iloc[0] != '':
        # print(brand['brand'])
        if len(meta_use[meta_use['brand'] == brand['brand'].iloc[0]]) >= 10:
          [recommend_cates.append(meta_use[meta_use['brand'] == brand['brand'].iloc[0]].iloc[gg]['asin']) for gg in range(0, 10)]
        else:
          [recommend_cates.append(meta_use[meta_use['brand'] == brand['brand'].iloc[0]].iloc[gg]['asin']) for gg in range(0, len(meta_use[meta_use['brand'] == brand['brand'].iloc[0]]))]

    recommendations[ii] = recommend_cates
    # recommendations = {ii : recommend_cates for ii in users}
  return recommendations


In [ ]:
def recommender2(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    for ii in users:
      recommend_cates = []
      recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == ii]['asin'].tolist())]['title'].tolist(), k)
      
      top_10 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]
      if len(recommend_cates) <= 10:
        [recommend_cates.append(top_10[gg]) for gg in range(0, 10-len(recommend_cates))]
      recommendations[ii] = recommend_cates
      # recommendations = {ii : recommend_cates for ii in users}
    # recommendations = {user: recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k) for user in users}
    return recommendations
    

In [ ]:
def recommender3(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    training_data = training_data[(training_data['DATE'] >= '2018-05-31') &  (training_data['DATE'] <= '2018-09-01')]
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    for ii in users:
      recommend_cates = []
      recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == ii]['asin'].tolist())]['title'].tolist(), k)
      
      top_10 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:10]
      if len(recommend_cates) <= 10:
        [recommend_cates.append(top_10[gg]) for gg in range(0, 10-len(recommend_cates))]
      recommendations[ii] = recommend_cates
 
    return recommendations
    

In [ ]:
import random
def recommender4(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    training_data = training_data[(training_data['DATE'] >= '2018-05-31') &  (training_data['DATE'] <= '2018-09-01')]
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    for ii in users:
      recommend_cates = []
      recommend_cates = recommend_items(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == ii]['asin'].tolist())]['title'].tolist(), k)
      
      top_15 = training_data.groupby('asin').size().sort_values(ascending=False).index.to_list()[:15]
      if len(recommend_cates) <= 10:
        random.shuffle(top_15)
        [recommend_cates.append(top_15[gg]) for gg in range(0, 10-len(recommend_cates))]
      recommendations[ii] = recommend_cates
 
    return recommendations
    

In [ ]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))
    score = total / len(ratings_testings)
    return score


In [ ]:
# # 包含全部的testing顧客
evaluate_result = {}
n = 0
R = [recommender, recommender1, recommender2, recommender3, recommender4] #, recommender5
for recommender in R: 
  ratings_by_user = {}
  ratings_by_user = recommender(ratings_trainings, users)
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])

result


,recommender_list,score
0,0,0.000000
1,1,0.000000
2,2,0.083051
3,3,0.133898
4,4,0.098305


In [ ]:
# # 只計算同時出現在training和testing的顧客
evaluate_result = {}
n = 0
R = [recommender, recommender1, recommender2, recommender3, recommender4] #, recommender5
for recommender in R: 
  ratings_by_user = {}
  # print(ratings_by_user)
  ratings_by_user = recommender(ratings_trainings, dic_user)
  evaluate_result[str(n)] = evaluate(ratings_testings_by_user, ratings_by_user)
  n += 1
result = pd.DataFrame(list(evaluate_result.items()), columns=['recommender_list', 'score'])
result

,recommender_list,score
0,0,0.110169
1,1,0.000000
2,2,0.083051
3,3,0.133898
4,4,0.088136
